In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

SENSOR_CSV = r"C:\Users\Dell\Downloads\Iot_part1.csv"
META_CSV   = r"C:\Users\Dell\Downloads\12k_trip_part_1.csv"

OUT_DIR = Path(r"C:\Users\Dell\Downloads")
OUT_DIR.mkdir(parents=True, exist_ok=True)

OUT_MERGED_CSV = OUT_DIR / "merged_timeseries_per_reading.csv"
OUT_MERGED_PARQUET = OUT_DIR / "merged_timeseries_per_reading.parquet"

print("Sensor file:", SENSOR_CSV)
print("Metadata file:", META_CSV)
print("Output (CSV):", OUT_MERGED_CSV)
print("Output (Parquet):", OUT_MERGED_PARQUET)


Sensor file: C:\Users\Dell\Downloads\Iot_part1.csv
Metadata file: C:\Users\Dell\Downloads\12k_trip_part_1.csv
Output (CSV): C:\Users\Dell\Downloads\merged_timeseries_per_reading.csv
Output (Parquet): C:\Users\Dell\Downloads\merged_timeseries_per_reading.parquet


# Load sensor-level và loại bỏ 2 cột không cần thiết

In [2]:
# parse_dates để có timestamp dạng datetime
sensor = pd.read_csv(SENSOR_CSV, parse_dates=['timestamp'], low_memory=False)

print("Cột sensor ban đầu:", sensor.columns.tolist())

# Xóa 2 cột speed_knots và failure_scenario nếu tồn tại
for col in ['speed_knots', 'failure_scenario']:
    if col in sensor.columns:
        sensor.drop(columns=[col], inplace=True)
        print(f"Đã xóa cột {col} khỏi sensor-level")

# Chuẩn hoá timestamp (tz-naive)
sensor['timestamp'] = pd.to_datetime(sensor['timestamp'], errors='coerce')
sensor['timestamp'] = sensor['timestamp'].dt.tz_localize(None)

# đảm bảo có cột trip_id
if 'trip_id' not in sensor.columns:
    raise RuntimeError("Không tìm thấy cột 'trip_id' trong file sensor-level. Kiểm tra file input.")

print("Số bản ghi sensor:", len(sensor))
print("Ví dụ 5 dòng:")
display(sensor.head(5))

Cột sensor ban đầu: ['timestamp', 'temp', 'humid', 'co2', 'light', 'class', 'trip_id', 'fruit_cate', 'latitude', 'longitude', 'speed_knots', 'Route', 'temperature_C', 'humidity_%', 'dew_point_C', 'pressure_hPa', 'wind_speed_kmh', 'precipitation_mm', 'failure_scenario']
Đã xóa cột speed_knots khỏi sensor-level
Đã xóa cột failure_scenario khỏi sensor-level
Số bản ghi sensor: 9010878
Ví dụ 5 dòng:


,timestamp,temp,humid,co2,light,class,trip_id,fruit_cate,latitude,longitude,Route,temperature_C,humidity_%,dew_point_C,pressure_hPa,wind_speed_kmh,precipitation_mm
0,2024-02-01 22:25:52,24.944977,89.257717,350.970178,20.473861,Good,TRIP_00030,Banana,10.74535,106.70465,VNSGN-USCHI,23.9,89.0,22.0,1012.3,8.4,0.0
1,2024-02-01 22:35:52,24.930127,88.937926,351.754231,17.989037,Good,TRIP_00030,Banana,10.74535,106.70465,VNSGN-USCHI,23.9,89.0,22.0,1012.3,8.4,0.0
2,2024-02-01 22:45:52,25.082836,88.535408,352.571613,20.079832,Good,TRIP_00030,Banana,10.74535,106.70465,VNSGN-USCHI,23.9,89.0,22.0,1012.3,8.4,0.0
3,2024-02-01 22:55:52,25.067485,88.786104,353.393199,19.468183,Good,TRIP_00030,Banana,10.74535,106.70465,VNSGN-USCHI,23.9,89.0,22.0,1012.3,8.4,0.0
4,2024-02-01 23:05:52,25.109032,88.616012,354.223067,20.142465,Good,TRIP_00030,Banana,10.74535,106.70465,VNSGN-USCHI,23.9,89.0,22.0,1012.3,8.4,0.0


# Load trip metadata và chọn 4 cột

In [4]:
meta = pd.read_csv(META_CSV, low_memory=False)

# Kiểm tra tên cột hiện có
print("Cột metadata:", meta.columns.tolist())

# Chọn các cột cần giữ lại
cols_keep = ['trip_id', 'trip_id_raw', 'expected_delay_min', 'weight', 'unit_quantity']
missing_cols = [c for c in cols_keep if c not in meta.columns]
if missing_cols:
    raise ValueError(f"Thiếu các cột bắt buộc trong metadata: {missing_cols}")

# Giữ đúng thứ tự cột
meta_sel = meta[cols_keep].copy()

# Chuẩn hoá kiểu dữ liệu
meta_sel['trip_id'] = meta_sel['trip_id'].astype(str).str.strip()
meta_sel['trip_id_raw'] = meta_sel['trip_id_raw'].astype(str).str.strip()

print("Số trip metadata:", len(meta_sel))
display(meta_sel.head(6))

Cột metadata: ['trip_id', 'carrier', 'start_time', 'end_time', 'duration_hours', 'origin', 'destination', 'product_id', 'unit_quantity', 'weight', 'service_level', 'priority', 'plant_code', 'expected_delay_min', 'target_temp_c', 'distance_km', 'trip_id_raw', 'fruit_cate', 'Route', 'tpt_days', 'start_month', 'start_year', 'year_month']
Số trip metadata: 3000


,trip_id,trip_id_raw,expected_delay_min,weight,unit_quantity
0,TRIP_01734,1447280295.7,4,0.522278,261
1,TRIP_142715,1447148658.7,2,2.354118,242
2,TRIP_90344,1447320530.7,9,11.560701,927
3,TRIP_53406,1447389167.7,8,4.179934,509
4,TRIP_19132,1447174185.7,2,110.480000,8709
5,TRIP_122058,1447148724.7,0,3.833742,295


# Kiểm tra sự khớp giữa trip_id trong sensor và metadata

In [5]:
sensor['trip_id'] = sensor['trip_id'].astype(str).str.strip()

sensor_trip_set = set(sensor['trip_id'].unique())
meta_trip_set   = set(meta_sel['trip_id'].unique())

in_sensor_not_meta = sorted(list(sensor_trip_set - meta_trip_set))[:10]
in_meta_not_sensor = sorted(list(meta_trip_set - sensor_trip_set))[:10]

print("Số trip unique trong sensor:", len(sensor_trip_set))
print("Số trip unique trong metadata:", len(meta_trip_set))
print("Ví dụ trip trong sensor nhưng KHÔNG trong metadata (max 10):", in_sensor_not_meta)
print("Ví dụ trip trong metadata nhưng KHÔNG trong sensor (max 10):", in_meta_not_sensor)

# Tỉ lệ sensor-trip có metadata
matched = len(sensor_trip_set & meta_trip_set)
print(f"Số trip khớp giữa 2 bảng: {matched} / {len(sensor_trip_set)} (sensor_unique)")


Số trip unique trong sensor: 3000
Số trip unique trong metadata: 3000
Ví dụ trip trong sensor nhưng KHÔNG trong metadata (max 10): []
Ví dụ trip trong metadata nhưng KHÔNG trong sensor (max 10): []
Số trip khớp giữa 2 bảng: 3000 / 3000 (sensor_unique)


# Thực hiện merge (left join) và sắp xếp theo trip + timestamp

In [6]:
merged = sensor.merge(meta_sel, on='trip_id', how='left', validate='m:1')  # many sensor rows -> one metadata row

# Sắp xếp để dễ nhìn: theo trip_id rồi timestamp
merged = merged.sort_values(['trip_id','timestamp']).reset_index(drop=True)

print("Kết quả merge shape:", merged.shape)
print("Số bản ghi có metadata missing (na in expected_delay_min):", merged['expected_delay_min'].isna().sum())
display(merged.head(8))

Kết quả merge shape: (9010878, 21)
Số bản ghi có metadata missing (na in expected_delay_min): 0


,timestamp,temp,humid,co2,light,class,trip_id,fruit_cate,latitude,longitude,...,temperature_C,humidity_%,dew_point_C,pressure_hPa,wind_speed_kmh,precipitation_mm,trip_id_raw,expected_delay_min,weight,unit_quantity
0,2024-02-01 22:25:52,24.944977,89.257717,350.970178,20.473861,Good,TRIP_00030,Banana,10.745350,106.704650,...,23.90,89.00,22.00,1012.3,8.400,0.0,1447257238.7,7,112.6,1044
1,2024-02-01 22:35:52,24.930127,88.937926,351.754231,17.989037,Good,TRIP_00030,Banana,10.745350,106.704650,...,23.90,89.00,22.00,1012.3,8.400,0.0,1447257238.7,7,112.6,1044
2,2024-02-01 22:45:52,25.082836,88.535408,352.571613,20.079832,Good,TRIP_00030,Banana,10.745350,106.704650,...,23.90,89.00,22.00,1012.3,8.400,0.0,1447257238.7,7,112.6,1044
3,2024-02-01 22:55:52,25.067485,88.786104,353.393199,19.468183,Good,TRIP_00030,Banana,10.745350,106.704650,...,23.90,89.00,22.00,1012.3,8.400,0.0,1447257238.7,7,112.6,1044
4,2024-02-01 23:05:52,25.109032,88.616012,354.223067,20.142465,Good,TRIP_00030,Banana,10.745350,106.704650,...,23.90,89.00,22.00,1012.3,8.400,0.0,1447257238.7,7,112.6,1044
5,2024-02-01 23:15:52,25.221654,88.947627,355.247677,19.474877,Good,TRIP_00030,Banana,10.745350,106.704650,...,23.90,89.00,22.00,1012.3,8.400,0.0,1447257238.7,7,112.6,1044
6,2024-02-01 23:25:52,24.926738,89.559712,355.970901,20.779193,Good,TRIP_00030,Banana,10.625067,106.802181,...,24.75,85.25,22.05,1013.2,12.625,0.0,1447257238.7,7,112.6,1044
7,2024-02-01 23:35:52,25.263376,89.186559,356.688357,19.613527,Good,TRIP_00030,Banana,10.625067,106.802181,...,24.75,85.25,22.05,1013.2,12.625,0.0,1447257238.7,7,112.6,1044


# Lưu kết quả (CSV & Parquet). Tùy chọn partition theo trip_id

In [7]:
merged.to_csv(OUT_MERGED_CSV, index=False, encoding='utf-8-sig')
print("Đã lưu CSV:", OUT_MERGED_CSV)

try:
    merged.to_parquet(OUT_MERGED_PARQUET, index=False)
    print("Đã lưu Parquet:", OUT_MERGED_PARQUET)
except Exception as e:
    print("Không lưu Parquet (thiếu pyarrow?):", e)
    print("Bạn vẫn có CSV để dùng.")

Đã lưu CSV: C:\Users\Dell\Downloads\merged_timeseries_per_reading.csv
Đã lưu Parquet: C:\Users\Dell\Downloads\merged_timeseries_per_reading.parquet


- (Tùy chọn) Tạo index MultiIndex (trip_id, timestamp) cho thao tác time-series thuận tiện

In [10]:
ts_indexed = merged.set_index(['trip_id','timestamp']).sort_index()
# ví dụ truy vấn nhanh: lấy 1 trip rồi resample theo 15 phút (nếu cần)
example_trip = merged['trip_id'].unique()[0]   
print("Ví dụ lấy trip:", example_trip)
sample_trip = ts_indexed.loc[example_trip]
print("Sample trip index shape:", sample_trip.shape)
display(sample_trip.head(8))

Ví dụ lấy trip: TRIP_00002
Sample trip index shape: (985, 19)


,temp,humid,co2,light,class,fruit_cate,latitude,longitude,Route,temperature_C,humidity_%,dew_point_C,pressure_hPa,wind_speed_kmh,precipitation_mm,trip_id_raw,expected_delay_min,weight,unit_quantity
timestamp,,,,,,,,,,,,,,,,,,,
2024-01-21 19:02:58,21.944977,85.257717,310.470178,14.473861,Good,Pineapple,20.844783,106.764010,VNHPH-CNSHA,16.4000,87.000,14.2,1017.200,24.40,0.0000,1447158014.7,1,87.94,3188
2024-01-21 19:12:58,21.930127,84.937926,310.754231,11.989037,Good,Pineapple,20.844783,106.764010,VNHPH-CNSHA,16.4000,87.000,14.2,1017.200,24.40,0.0000,1447158014.7,1,87.94,3188
2024-01-21 19:22:58,22.082836,84.535408,311.071613,14.079832,Good,Pineapple,20.844783,106.764010,VNHPH-CNSHA,16.4000,87.000,14.2,1017.200,24.40,0.0000,1447158014.7,1,87.94,3188
2024-01-21 19:32:58,22.067485,84.786104,311.393199,13.468183,Good,Pineapple,20.844783,106.764010,VNHPH-CNSHA,16.4000,87.000,14.2,1017.200,24.40,0.0000,1447158014.7,1,87.94,3188
2024-01-21 19:42:58,22.109032,84.616012,311.723067,14.142465,Good,Pineapple,20.844783,106.764010,VNHPH-CNSHA,16.4000,87.000,14.2,1017.200,24.40,0.0000,1447158014.7,1,87.94,3188
2024-01-21 19:52:58,22.221654,84.947627,312.247677,13.474877,Good,Pineapple,20.844783,106.764010,VNHPH-CNSHA,16.4000,87.000,14.2,1017.200,24.40,0.0000,1447158014.7,1,87.94,3188
2024-01-21 20:02:58,21.926738,85.559712,312.470901,14.779193,Good,Pineapple,20.910585,106.850366,VNHPH-CNSHA,15.6875,85.875,13.3,1016.275,23.25,0.0125,1447158014.7,1,87.94,3188
2024-01-21 20:12:58,22.263376,85.186559,312.688357,13.613527,Good,Pineapple,20.910585,106.850366,VNHPH-CNSHA,15.6875,85.875,13.3,1016.275,23.25,0.0125,1447158014.7,1,87.94,3188
